In [ ]:
import pandas as pd
import torch
import torchdms
import torchdms.data
import torchdms.model
from torchdms.analysis import Analysis

%matplotlib notebook

In [ ]:
[aa_func_scores, wtseq] = torchdms.data.from_pickle_file("../_ignore/aa_func_scores_and_wtseq.pkl")
[aa_func_scores, wtseq] = torchdms.data.from_pickle_file("../_ignore/gb1.pkl")

In [ ]:
test_partition, partitioned_train_data = torchdms.data.partition(aa_func_scores)
test_data = torchdms.data.bmapplus_of_aa_func_scores(test_partition, wtseq)
train_data_list = [
    torchdms.data.bmapplus_of_aa_func_scores(train_data_partition, wtseq) 
    for train_data_partition in partitioned_train_data]

In [ ]:
model = torchdms.model.SingleSigmoidNet(input_size=test_data.binarylength, hidden1_size=1)
analysis = Analysis(model, train_data_list)

In [ ]:
%%time

criterion = torch.nn.MSELoss()
losses = analysis.train(criterion, 3)
pd.Series(losses).plot()

In [ ]:
results = analysis.evaluate(test_data)
corr = results.corr().iloc[0,1]
results["n_aa_substitutions"] = test_data.n_aa_substitutions
ax = results.plot.scatter(x="Observed", y="Predicted", c = results["n_aa_substitutions"], cmap='viridis')
ax.text(0, 0.95*max(results["Predicted"]), f"corr = {corr:.3f}")